In [8]:
import numpy as np
import dask.array as da
from dask.distributed import Client, as_completed
from scipy import sparse

# PLEASE
Run only what you need. Thanks

## Simultaneous max and min

In [ ]:
def minmax(x):
    y = np.max(x[:, None, :] * np.array([-1, 1])[:, None, None], axis=2)
    #y[:, 0] *= -1
    return y

def minmax2(x):
    return np.concatenate([np.min(x, axis=1)[:,None], np.max(x, axis=1)[:,None]], axis=1)

In [ ]:
x = np.arange(10000000).reshape(-1,50,2)

#assert np.allclose(minmax(x), minmax2(x))

%timeit minmax(x)
%timeit minmax2(x)

## `np.nan` is as performant as a shorter matrix?

In [ ]:
x = np.arange(1000000).reshape(-1,2)
print(x.shape)

%timeit x + 2 - np.array([2,20])[None]

y = np.vstack([x, np.full((500000, 2), np.nan)])
print(y.shape)

%timeit y + 2 - np.array([2,20])[None]

z = np.vstack([x, x])
print(z.shape)

%timeit z + 2 - np.array([2,20])[None]

## Best way to achieve all `False` in NumPy

In [ ]:
x = np.full((10000,1000), False)
x[2,5] = True
x[1000,10] = True

In [ ]:
%timeit np.all(np.logical_not(x), axis=1)
%timeit np.logical_not(np.any(x, axis=1))
%timeit ~np.any(x, axis=1)
%timeit (~x).all(axis=1)

## Is `map_blocks` better than the standard operation?

In [ ]:
y = da.from_array(np.ones((1000,1000,200)))

print('Simple subtraction')
%timeit (y - 10).compute()
%timeit y.map_blocks(lambda x: x - 10).compute()

print('Some references to y')
%timeit (y - 10 + 2*y).compute()
%timeit y.map_blocks(lambda x: x - 10 + 2*x).compute()

print('Multiple references to y')
%timeit (y + 2*y - 3*y + 4*y - 5*y).compute()
%timeit y.map_blocks(lambda x: x + 2*x - 3*x + 4*x - 5*x).compute()

print('NumPy function')
%timeit (np.exp(y+2)).compute()
%timeit y.map_blocks(lambda x: np.exp(x)).compute()

## Is `map_blocks` better/worse than the corresponding Dask function?

In [ ]:
y = da.from_array(np.ones((1000,1000,20)) + 0.5)

%timeit y.map_blocks(lambda x: np.floor(x)).compute()
%timeit da.floor(y).compute()

## Does `np.unique` work?

In [ ]:
y = da.from_array(np.arange(10000))
da.unique(y, return_index=True)[1].compute()

## What about `argsort`?

In [ ]:
y = da.from_array(np.arange(10000))
y = y[da.argsort(y)]

## What's the best way to create a matrix such that different submatrices are filled with different rules?

In [ ]:
def fill1():
    x = np.zeros((9900, 100), dtype=float)
    for i in range(1,100):
        x[(i-1)*100:i*100] = i*np.arange(i*100-100,i*100)[::-1]
    return x

def fill2():
    return np.vstack([
        np.repeat(i*np.arange(i*100-100,i*100)[::-1][None], repeats=100, axis=0) for i in range(1,100)
    ])


%timeit fill1()
%timeit fill2()

## What's the best way to fill a matrix taking values from a vector according to some indexing?

In [ ]:
from itertools import chain

def generate_binified_points_matrix(pts, indexes, bins_size):
    zs = np.zeros((bins_size - 1, pts.shape[1]))
    for idx in range(len(indexes)):
        idx = indexes[idx]
        yield pts[idx], zs[: bins_size - len(idx)]
        
def bins1(pts, indexes_inside_bins, biggest_bin):
    return np.vstack(
            tuple(chain.from_iterable(
                generate_binified_points_matrix(
                    pts, indexes_inside_bins, biggest_bin
                )
            ))
        ).reshape(len(indexes_inside_bins), biggest_bin, pts.shape[1])

def bins2(pts, indexes_inside_bins, biggest_bin):
    nbins = len(indexes_inside_bins)
    bins = np.zeros((nbins, biggest_bin, pts.shape[1]))
    for bin_idx in range(nbins):
        ps = pts[indexes_inside_bins[bin_idx]]
        bins[bin_idx, : len(ps)] = ps
    return bins

In [ ]:
for n_components in (1,2,3,5,10,100):
    pts = np.random.rand(10000, n_components)
    indexes_inside_bins = np.arange(10000)[::-1]
    split_sizes = [100,200,400,50, 50, 1000, 1500, 500, 900, 100, 500, 1000, 500, 100, 200, 200, 
                   500, 1000, 1, 1, 1, 7, 80, 5]
    indexes_inside_bins = np.split(indexes_inside_bins, np.cumsum(split_sizes))

    M = np.max(split_sizes)
    
    print('n components = {}'.format(n_components))
    %timeit bins1(pts, indexes_inside_bins, M)
    %timeit bins2(pts, indexes_inside_bins, M)

## Incidence of an `if` statement in a for loop

In [ ]:
def f1():
    for i in range(100000):
        if i == -1:
            print('this will never happen')
        x = i*2-1
        
def f2():
    for i in range(100000):
        x = i*2-1
        
%timeit f1()
%timeit f2()

## Is `np.nansum` faster?

In [ ]:
x = np.zeros((10000, 1000))
x[1000:2000] = 4
x[6000:8000] = 12

%timeit np.sum(x)
print(np.sum(x))

y = np.full((10000, 1000), np.nan)
y[1000:2000] = 4
y[6000:8000] = 12

%timeit np.nansum(y)
print(np.nansum(y))

## Fastest way to compute Hankel matrix

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view as swv

def numpy_version(X, d):
    n_ho_snapshots = X.shape[1] - d + 1

    idxes = np.repeat(
        np.arange(d)[None], repeats=n_ho_snapshots, axis=0
    )
    idxes += np.arange(n_ho_snapshots)[:,None]
    return X.T[idxes].reshape(-1, X.shape[0] * d).T

def numpy_version2(X, d):
    n_ho_snapshots = X.shape[1] - d + 1

    idxes = np.repeat(
        np.arange(d)[None], repeats=n_ho_snapshots, axis=0
    )
    idxes += np.arange(n_ho_snapshots)[:,None]
    return np.swapaxes(X[:,idxes],1,2).reshape(-1, X.shape[0] * d, order='F')

def numpy_version3(X, d):
    n_ho_snapshots = X.shape[1] - d + 1

    idxes = np.repeat(
        np.arange(d)[None], repeats=n_ho_snapshots, axis=0
    )
    idxes += np.arange(n_ho_snapshots)[:,None]
    return np.swapaxes(X[:,idxes],0,1).reshape(-1, X.shape[0] * d).T

def numpy_version3(X, d):
    n_ho_snapshots = X.shape[1] - d + 1

    idxes = np.repeat(
        np.arange(d)[None], repeats=n_ho_snapshots, axis=0
    )
    idxes += np.arange(n_ho_snapshots)[:,None]
    idxes = idxes.flatten()
    return np.swapaxes(X[:,idxes],0,1).reshape(-1, X.shape[0] * d).T

def numpy_version4(X, d):
    n_ho_snapshots = X.shape[1] - d + 1

    idxes = np.repeat(
        np.arange(d)[None], repeats=n_ho_snapshots, axis=0
    )
    idxes += np.arange(n_ho_snapshots)[:,None]
    return X[:, idxes.flatten()].reshape(X.shape[0] * d, -1, order='F')

def stride_version(X, d):
    n_ho_snapshots = X.shape[1] - d + 1
    return swv(X, (X.shape[0], d))[0].reshape(n_ho_snapshots, -1, order='F').T

def stride_version2(X, d):
    n_ho_snapshots = X.shape[1] - d + 1
    return swv(X.T, (d, X.shape[0]))[:,0].reshape(n_ho_snapshots, -1).T

def python_list_version(X, d):
    return np.concatenate(
            [X[:, i : X.shape[1] - d + i + 1] for i in range(d)],
            axis=0,
    )

In [ ]:
X = np.ones((500, 1000))
d = 10

%timeit numpy_version(X,d)
%timeit numpy_version2(X,d)
%timeit numpy_version3(X,d)
%timeit numpy_version4(X,d)
%timeit stride_version(X,d)
%timeit stride_version2(X,d)
%timeit python_list_version(X,d)

print('-----')

X = np.ones((500, 1000))
d = 100

%timeit numpy_version(X,d)
%timeit numpy_version2(X,d)
%timeit numpy_version3(X,d)
%timeit numpy_version4(X,d)
%timeit stride_version(X,d)
%timeit stride_version2(X,d)
%timeit python_list_version(X,d)

print('-----')

X = np.ones((500, 10000))
d = 100

%timeit numpy_version(X,d)
%timeit numpy_version2(X,d)
%timeit numpy_version3(X,d)
%timeit numpy_version4(X,d)
%timeit stride_version(X,d)
%timeit stride_version2(X,d)
%timeit python_list_version(X,d)

## Transpose and `swapaxes`

In [ ]:
def swap(X):
    if X.ndim == 3:
        return np.swapaxes(X, 1,2).T
    elif X.ndim == 4:
        return np.swapaxes(np.swapaxes(np.swapaxes(X, 0,2), 2,3), 0,1)

def swap2(X):
    r = np.arange(X.ndim)
    return np.moveaxis(X, r, np.roll(r, 1))

shapes = [(20000,100,100,100), (200,10000,100,100), (200,100,10000,100), (200,100,100,10000)]
for shape in shapes:
    X = np.empty(shape)
    print(X.shape)
    %timeit swap(X)
    %timeit swap2(X)
    assert swap(X).shape == swap2(X).shape
    del X

## Is using the `out` parameter faster?

In [ ]:
x = np.arange(100000000).reshape(100, -1)
%timeit y = np.clip(x, 100, 101)
%timeit np.clip(x, 100, 101, out=x)

## Shared memory in Dask?

In [ ]:
client = Client(processes=False)

In [ ]:
def fill_sm(idx, matrix):
    matrix[idx] = idx

def fill(idx, n_columns):
    return np.repeat([idx], repeats=n_columns)[None]

def fill_with_as_completed(rows, columns):
    def fill(idx):
        return idx, np.repeat([idx], repeats=columns)
    
    matrix=np.zeros((rows, columns))
    futures = client.map(fill, range(rows))
    for _, (row_idx, row) in as_completed(futures, with_results=True):
        matrix[row_idx] = row

M = 100
N = 10000

%timeit client.gather(client.map(fill_sm, range(M), matrix=np.zeros((M,N))));
%timeit da.vstack([fill(i, N) for i in range(M)])
%timeit fill_with_as_completed(M,N)

## Filling a sparse matrix

In [6]:
def group_by(a):
    a = a[a[:, 0].argsort()]
    return np.split(a[:, 1], np.unique(a[:, 0], return_index=True)[1][1:])

# P is the number of covariates
# k is the number of levels per covariate
# n is the dimension of the sample
def random_sample(P, n, k, probability_generator=None):
    if probability_generator is None:

        def probability_generator(arr):
            return np.ones_like(arr) / len(arr)

    ls = [None for p in range(P)]
    for p in range(P):
        arr = np.arange(k[p])
        probabilities = probability_generator(arr)

        # randomly select a level for each control value
        choice = np.random.choice(arr, size=n, p=probabilities)
        indexed_choices = np.hstack([choice[:, None], np.arange(n)[:, None]])
        gb = group_by(indexed_choices)

        un = np.unique(choice)
        # add missing levels
        i = 0
        for u in un:
            for j in range(i, u):
                gb.insert(i, [])
            i = u + 1
        for j in range(i, k[p]):
            gb.append([])

        ls[p] = gb
    return ls

def random_Lprime(P, n_prime, k, probability_generator=None):
    return random_sample(P, n_prime, k, probability_generator=probability_generator)

def random_l(P, n, k, probability_generator=None):
    ls = random_sample(P, n, k, probability_generator=probability_generator)
    return list(map(lambda l: list(map(len, l)), ls))

# with P=2
def generate_problems(n, n_prime, k1, k2, probability_generator=None):
    l = random_l(2, n, (k1, k2), probability_generator=probability_generator)
    L_prime = random_Lprime(
        2, n_prime, (k1, k2), probability_generator=probability_generator
    )
    return l, L_prime

In [5]:
def sparse_method(L_prime, k, n_prime):
    csk = np.concatenate([[0], np.cumsum(k[:-1])])
    shape = (csk[-1] + k[-1], n_prime)
    
    row_ind = np.concatenate([np.repeat(csk[p] + i, len(L_prime[p][i])) 
                              for p in range(len(L_prime)) for i in range(k[p])])
    col_ind = np.concatenate([L_prime[p][i] for p in range(len(L_prime)) for i in range(k[p])])
    data = np.ones_like(row_ind, dtype=int)
    
    return sparse.coo_matrix((data, (row_ind, col_ind)), shape=shape)

def dense_method(L_prime, k, n_prime):
    A = np.zeros((sum(k), n_prime), dtype=int)
    current_row = 0
    for p in range(len(L_prime)):
        Lp_prime = L_prime[p]
        for i in range(k[p]):
            A[current_row, Lp_prime[i]] = 1
            current_row += 1
    return A

In [ ]:
k = [100, 50]
n, n_prime = 10000, 100000
l, L_prime = generate_problems(n, n_prime, *k)

A_sparse = sparse_method(L_prime, k, n_prime).toarray()
A_dense = dense_method(L_prime, k, n_prime)
np.testing.assert_almost_equal(A_sparse, A_dense)

%timeit sparse_method(L_prime, k, n_prime)
%timeit sparse_method(L_prime, k, n_prime).toarray()
%timeit sparse_method(L_prime, k, n_prime).tocsc()
%timeit sparse_method(L_prime, k, n_prime).tocsr()
print('----')
%timeit sparse_method_csc(L_prime, k, n_prime)
%timeit sparse_method_csr(L_prime, k, n_prime)
print('----')
%timeit dense_method(L_prime, k, n_prime)

## Computing matrix `U` 
### From the paper *Network flow methods for the minimum covariate imbalance problem*

In [3]:
def sparse_compute_U(A, k0):
    A1 = A[:k0]
    A2 = A[k0:]
    return A1.dot(A2.T).toarray()

def sparse_compute_U2(A, k0):
    positive_A = (A > 0).toarray()
    A1 = positive_A[: k0]
    A2 = positive_A[k0 :]
    return np.count_nonzero(
        np.logical_and(A1[:, None], A2[None]), 
        axis=-1)

def dense_compute_U(A, k0):
    A1 = A[: k0] > 0
    A2 = A[k0 :] > 0
    return np.count_nonzero(
        np.logical_and(A1[:, None], A2[None]), 
        axis=-1)

In [9]:
k = [100, 200]
n, n_prime = 10000, 100000
l, L_prime = generate_problems(n, n_prime, *k)

A_sparse = sparse_method(L_prime, k, n_prime).tocsr()
A_dense = dense_method(L_prime, k, n_prime)

%timeit sparse_compute_U(A_sparse, k[0])
%timeit sparse_compute_U2(A_sparse, k[0])
%timeit dense_compute_U(A_dense, k[0])

7.02 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4 s ± 319 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.47 s ± 544 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
from time import time
from itertools import product

def benchmark(func, A, k):
    start = time()
    func(A, k)
    return time() - start

def generate_sparsedense_A(k, nprime):
    n = 10
    k = (k,k)
    l, L_prime = generate_problems(n, nprime, *k)

    A_sparse = sparse_method(L_prime, k, n_prime).tocsr()
    A_dense = dense_method(L_prime, k, n_prime)
    
    return A_sparse, A_dense

ks = [10, 100, 1000, 10000]
nprimes = [10, 100, 1000, 10000]
times = 5

funcs = [dense_compute_U, sparse_compute_U2, sparse_compute_U]
is_sparse = [False, True, True]
times_matrix = np.zeros((len(funcs), len(ks), len(nprimes)), dtype=float)
for k, k_idx, nprime, nprime_idx in product(ks, range(len(ks)), nprimes, range(len(nprimes))):
    A_sparse, A_dense = generate_problem(k, nprime)
    
    for func_idx, func in enumerate(funcs):
        A = A_sparse if is_sparse[func_idx] else A_dense
        for _ in range(times):
            times_matrix[func_idx, k_idx, nprime_idx] += benchmark(func, A, k)
times_matrix /= times

KeyboardInterrupt: 

## Generating a 2D grid

In [7]:
def grid_1(N):
    x = np.linspace(-5, 5, N)
    y = np.linspace(-5, 5, N)
    xx, yy = np.meshgrid(x, y)
    return np.dstack(np.meshgrid(x,y))

def grid_2(N):
    step = 10 / (N - 1)
    return (np.mgrid[:N, :N] * step - 5).T

np.testing.assert_allclose(grid_1(int(1.e2)), grid_2(int(1.e2)))

%timeit grid_1(int(1.e2))
%timeit grid_2(int(1.e2))

252 µs ± 6.93 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
137 µs ± 8.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Is `slice` faster than `[:]`?

In [14]:
x = np.arange(100000000).reshape(500, 100, -1)
print('1D')
%timeit x[100:498]
%timeit x[slice(100,498)]
print('2D')
%timeit x[100:498, 8:90]
%timeit x[slice(100,498), slice(8,90)]
print('3D')
%timeit x[100:498, 8:90, 1000:1800]
%timeit x[slice(100,498), slice(8,90), slice(1000,1800)]

1D
299 ns ± 15.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
409 ns ± 5.45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
2D
456 ns ± 8.66 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
702 ns ± 4.94 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
3D
550 ns ± 18.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
887 ns ± 7.51 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Is `np.empty` faster than `np.zeros`?

In [15]:
%timeit np.empty((10000, 100, 10))
%timeit np.zeros((10000, 100, 10), dtype=int)
%timeit np.zeros((10000, 100, 10), dtype=float)

1.51 µs ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
17.4 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
17.3 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## What if we have a mask of values to initialize to something?

In [18]:
u = np.random.rand(1000000, 2)
point = np.array([0.75, 0.75])
d = 0.3

def function(x):
    return np.power(x, 2)

distances = np.linalg.norm(u - point[None], axis=-1)
nearby = distances < d

def with_empty():
    nnearby = np.logical_not(nearby)
    
    mapped_distance = np.empty_like(distances, dtype=float)
    mapped_distance[nearby] = function(distances[nearby])
    mapped_distance[nnearby] = 0
    return mapped_distance

def without_empty():
    mapped_distance = np.zeros_like(distances, dtype=float)
    mapped_distance[nearby] = function(distances[nearby])
    return mapped_distance

np.testing.assert_allclose(with_empty(), without_empty())

%timeit with_empty()
%timeit without_empty()

26.5 ms ± 881 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
22.5 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Should we recycle the distance matrix?

In [34]:
def recycle(dst):
    dst[nearby] = function(dst[nearby])
    dst[np.logical_not(nearby)] = 0
    return dst

def waste(dst):
    mapped_distance = np.zeros_like(dst, dtype=float)
    mapped_distance[nearby] = function(dst[nearby])
    return mapped_distance

distances = np.linalg.norm(u - point[None], axis=-1)
np.testing.assert_allclose(waste(distances), recycle(distances))
distances = np.linalg.norm(u - point[None], axis=-1)

%timeit recycle(distances)
%timeit waste(distances)

20.5 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
16.2 ms ± 99.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Is writing in slices more efficient?

In [3]:
x = np.zeros((100000, 1000))

%timeit x[:10000] = 4

idxes = np.arange(10000, 20000)
%timeit x[idxes] = 4

9.75 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.2 ms ± 9.68 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Periodic write

In [ ]:
def with_sliding_window(arr, write, lower_bounds, upper_bounds):
    pass